In [ ]:
!pip install transformers datasets optuna tensorboard
!pip install accelerate -U
!pip install transformers -U
print("Accelerate version:", accelerate.__version__)
print("Transformers version:", transformers.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731

NameError: name 'accelerate' is not defined

#Preprocess

In [ ]:
import pandas as pd

sum_df = pd.read_csv('/content/drive/MyDrive/TrainingBart/summary_dataset.csv')
sum_df.head()
ori_df = pd.read_csv('/content/drive/MyDrive/TrainingBart/earning_calls_transcript_1k.csv')
ori_df.head()

,Exchange,Symbol,Year,EarningCallTranscript
0,NMS,MSFT,2021,"Operator: Greetings, and welcome to the Micros..."
1,NMS,MSFT,2021,"Operator: Greetings, and welcome to the Micros..."
2,NMS,MSFT,2021,"Operator: Greetings, and welcome to the Micros..."
3,NMS,MSFT,2021,"Operator: Greetings, and welcome to the Micros..."
4,NMS,MSFT,2022,"Operator: Greetings, and welcome to the Micros..."


In [ ]:
ori_df = ori_df.head(1000)
ori_df.index = sum_df.index
sum_df['EarningCallTranscript'] = ori_df['EarningCallTranscript']
sum_df.info()


sum_df.to_csv("final_dataset.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Exchange               1000 non-null   object
 1   Symbol                 1000 non-null   object
 2   Year                   1000 non-null   int64 
 3   EarningCallTranscript  1000 non-null   object
 4   Summary                1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB


Hyperparams Tuning

In [ ]:
import optuna
from transformers import BartTokenizer, BartForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers.integrations import TensorBoardCallback
from datasets import Dataset, DatasetDict
import pandas as pd
import os

In [ ]:
sum_df = pd.read_csv('/content/final_dataset.csv')

dataset = Dataset.from_pandas(sum_df)
dataset = dataset.rename_column("Summary", "target")
dataset = dataset.rename_column("EarningCallTranscript", "source")

In [ ]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

In [ ]:
def chunk_text(text, chunk_size=1024):
    words = text.split()
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

In [ ]:
def tokenize_function(examples):
    input_ids = []
    attention_mask = []
    labels = []

    for source_text, target_text in zip(examples['source'], examples['target']):
        source_text = ' '.join(source_text) if isinstance(source_text, list) else source_text
        target_text = ' '.join(target_text) if isinstance(target_text, list) else target_text

        source_chunks = chunk_text(source_text, chunk_size=1024)
        target_chunks = chunk_text(target_text, chunk_size=1024)

        for source_chunk, target_chunk in zip(source_chunks, target_chunks):
            model_inputs = tokenizer(source_chunk, max_length=1024, truncation=True, padding="max_length")
            labels_chunk = tokenizer(target_chunk, max_length=1024, truncation=True, padding="max_length")

            input_ids.append(model_inputs['input_ids'])
            attention_mask.append(model_inputs['attention_mask'])
            labels.append(labels_chunk['input_ids'])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }

# Tokenize dataset
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=8,
    remove_columns=dataset.column_names,
    num_proc=4
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train_test_split = tokenized_datasets.train_test_split(test_size=0.1)
tokenized_datasets = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

In [ ]:
log_dir = "./logs"
os.makedirs(log_dir, exist_ok=True)

In [ ]:
def objective(trial):
    # Define the hyperparameter search space
    args = Seq2SeqTrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True),
        per_device_train_batch_size=trial.suggest_int("per_device_train_batch_size", 2, 4),  # Reduced batch size range
        per_device_eval_batch_size=trial.suggest_int("per_device_eval_batch_size", 2, 4),   # Reduced batch size range
        weight_decay=trial.suggest_float("weight_decay", 0.01, 0.1),
        save_total_limit=3,
        num_train_epochs=trial.suggest_int("num_train_epochs", 3, 6),  # Adjusted epochs range
        predict_with_generate=True,
        fp16=True,
        logging_dir=log_dir,
        logging_steps=10,
        report_to=["tensorboard"],
        gradient_accumulation_steps=trial.suggest_int("gradient_accumulation_steps", 1, 4)  # Added gradient accumulation
    )

    model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        callbacks=[TensorBoardCallback()]
    )

    trainer.train()
    eval_results = trainer.evaluate()

    return eval_results["eval_loss"]

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

best_params = study.best_params
print("Best hyperparameters: ", best_params)

[I 2024-05-23 04:50:45,715] A new study created in memory with name: no-name-e64e7dca-1f21-4b36-be42-7f14ad6cbbbc
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

You are adding a <class 'transformers.integrations.integration_utils.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback


Epoch,Training Loss,Validation Loss
1,2.263600,1.874167
2,1.710000,1.515134
3,1.440400,1.479219
4,1.561700,1.464103
5,1.453600,1.458783
6,1.394000,1.455372


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


...

[I 2024-05-23 08:33:29,048] Trial 18 finished with value: 1.4751839637756348 and parameters: {'learning_rate': 3.0366035468605182e-05, 'per_device_train_batch_size': 4, 'per_device_eval_batch_size': 4, 'weight_decay': 0.07963122155915037, 'num_train_epochs': 3, 'gradient_accumulation_steps': 1}. Best is trial 5 with value: 1.4399908781051636.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
You are adding a <class 'transformers.integrations.integration_utils.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback


Epoch,Training Loss,Validation Loss
1,1.657800,1.550954
2,1.550900,1.473688
3,1.391200,1.452976
4,1.529500,1.445122


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


[I 2024-05-23 08:44:07,177] Trial 19 finished with value: 1.4451216459274292 and parameters: {'learning_rate': 4.06594428455066e-05, 'per_device_train_batch_size': 2, 'per_device_eval_batch_size': 3, 'weight_decay': 0.09110278428144425, 'num_train_epochs': 4, 'gradient_accumulation_steps': 2}. Best is trial 5 with value: 1.4399908781051636.


Best hyperparameters:  {'learning_rate': 2.8023652601511377e-05, 'per_device_train_batch_size': 2, 'per_device_eval_batch_size': 3, 'weight_decay': 0.08233597332385283, 'num_train_epochs': 4, 'gradient_accumulation_steps': 1}


In [ ]:
# Use the best hyperparameters for the final training
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=best_params["learning_rate"],
    per_device_train_batch_size=best_params["per_device_train_batch_size"],
    per_device_eval_batch_size=best_params["per_device_eval_batch_size"],
    weight_decay=best_params["weight_decay"],
    save_total_limit=3,
    num_train_epochs=best_params["num_train_epochs"],
    predict_with_generate=True,
    fp16=True,
    logging_dir=log_dir,
    logging_steps=10,
    report_to=["tensorboard"]
)

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    callbacks=[TensorBoardCallback()]
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
You are adding a <class 'transformers.integrations.integration_utils.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.823400,1.520738
2,1.661000,1.465809
3,1.307800,1.446562
4,1.451400,1.440279


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=1800, training_loss=1.7487749926249185, metrics={'train_runtime': 804.1808, 'train_samples_per_second': 4.477, 'train_steps_per_second': 2.238, 'total_flos': 7801576567603200.0, 'train_loss': 1.7487749926249185, 'epoch': 4.0})

In [ ]:
results = trainer.evaluate()
print(results)

{'eval_loss': 1.440279245376587, 'eval_runtime': 3.884, 'eval_samples_per_second': 25.746, 'eval_steps_per_second': 8.754, 'epoch': 4.0}


In [ ]:
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./finetuned_model/tokenizer_config.json',
 './finetuned_model/special_tokens_map.json',
 './finetuned_model/vocab.json',
 './finetuned_model/merges.txt',
 './finetuned_model/added_tokens.json')